<a href="https://colab.research.google.com/github/louispaulet/Classfication_and_Representation_Learning_course/blob/main/Food_Optimisation_Problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from scipy.optimize import linprog
import numpy as np
#protein | fat | fibre | salt
Chicken = [0.1, 0.08, 0.001, 0.002]
Beef = [0.2, 0.1, 0.005, 0.005]
Mutton = [0.15, 0.11, 0.003, 0.007]
Rice = [0.0, 0.01, 0.1, 0.008]
Wheat = [0.04, 0.01, 0.15, 0.0]
Gel = [0.0, 0.0, 0.0, 0.0]

pChicken = 0.013
pBeef = 0.008
pMutton = 0.010
pRice = 0.002
pWheat = 0.005
pGel = 0.001

prices = np.array([pChicken, pBeef, pMutton, pRice, pWheat, pGel])


Nutrition_mat = [Chicken, Beef, Mutton, Rice, Wheat, Gel]
Nutrition_mat = np.array(Nutrition_mat) #convert to numpy array so indexes work properly
Nutrition_mat

array([[0.1  , 0.08 , 0.001, 0.002],
       [0.2  , 0.1  , 0.005, 0.005],
       [0.15 , 0.11 , 0.003, 0.007],
       [0.   , 0.01 , 0.1  , 0.008],
       [0.04 , 0.01 , 0.15 , 0.   ],
       [0.   , 0.   , 0.   , 0.   ]])

In [ ]:
#function to minimize
#min(qC*pChicken+qB*pBeef+qM*pMutton+qR*pRice+qW*pWheat+qG*pGel)
#Quantities_Vector = [qC, qB, qM, qR, qW, qG]
#constraints
#sum(Quantities_Matrix*Nutrition_mat[:,0] > 8
#sum(Quantities_Matrix*Nutrition_mat[:,0] > 6
#sum(Quantities_Matrix*Nutrition_mat[:,0] > 2
#sum(Quantities_Matrix*Nutrition_mat[:,0] < 0.4
#sum(Quantities_Matrix) = 100

obj = [pChicken, pBeef, pMutton, pRice, pWheat, pGel]

#we multiply the "greater than" conditions by "-1" because scipy can only deal "inferior than" conditions 
lhs_ineq = np.array([
         -1 * Nutrition_mat[:, 0],
         -1 * Nutrition_mat[:, 1],
         -1 * Nutrition_mat[:, 2],
         Nutrition_mat[:, 3]
])

#same for the other side of the inequality
rhs_ineq = np.array([-8, -6, -2, 0.4])

lhs_eq = np.array([[1, 1, 1, 1, 1, 1]])
rhs_eq = [100]

In [ ]:
#we cannot make a recipe with negative food quantities so all constraints are positive
bnd = [(0, float("inf")), # Bounds of qChicken
       (0, float("inf")), # Bounds of qBeef
       (0, float("inf")), # Bounds of qMutton
       (0, float("inf")), # Bounds of qRice
       (0, float("inf")), # Bounds of qWheat
      (0, float("inf"))]  # Bounds of qGel

In [ ]:
 #execute linear programming
 opt = linprog(c=obj, A_ub=lhs_ineq, b_ub=rhs_ineq,
               A_eq=lhs_eq, b_eq=rhs_eq, bounds=bnd,
               method="revised simplex")
 

 opt

     con: array([-1.42108547e-14])
     fun: 0.531938514090521
 message: 'Optimization terminated successfully.'
     nit: 4
   slack: array([ 3.77796755e+00,  1.77635684e-15, -2.22044605e-16, -5.55111512e-17])
  status: 0
 success: True
       x: array([ 0.        , 58.41161401,  0.        , 13.49274125,  2.3911187 ,
       25.70452605])

In [ ]:
#checking that the solution satisfies the constraints
quantities = np.array(opt.x)
print ("price :" + str(round(np.sum(quantities * prices), 3)) + " | total weight : " + str(round(np.sum(quantities), 3)))

nutritional_values_of_finished_product = np.sum(np.transpose([quantities]) * Nutrition_mat, 0) #axis=0 to sum the rows over each column
nutrition_labels = ["Protein", "Fat", "Fibre", "Salt"]

print("\nNutritional values:")
i = 0
for nutrition_label in nutrition_labels:
  print("- " + nutrition_label + " : " + str(round(nutritional_values_of_finished_product[i], 3)))
  i += 1

price :0.532 | total weight : 100.0

Nutritional values:
- Protein : 11.778
- Fat : 6.0
- Fibre : 2.0
- Salt : 0.4
